# Create tables for product sales

We start importing the relevant modules and connect to the database callled `sales.db`

In [3]:
import sqlite3
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

Now we clean the database if there is any already table in there

In [6]:
# Querying for tables in the database
cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
tables = cursor.fetchall()  

# Dropping all tables in the database
if tables:
    print("Tables in the database:")
    for table in tables:
        print(table[0])
        cursor.execute("""DROP TABLE IF EXISTS {}""".format(table[0]))
else:
    print("No tables found in the database.")
print("Database cleared.")

Tables in the database:
produtos
Database cleared.


## Create table structure

We will create three tables `products` and `clients` which will store information about each entity and the `sales` will estabilish the connection between them.

products: Information about the products. 
* id_product (primary key)
* product_name
* category
* price

sales: Details of each sales transaction.
* id_sale (primary key)
* id_product (foreign key - products table)
* sale_date
* quantity
* id_client (foreign key - clients table)

clients: Informations about clients.
* id_client (primary key)
* client_name
* city
* state

In [9]:
cursor.execute("""CREATE TABLE IF NOT EXISTS products (
    id_product INTEGER PRIMARY KEY,
    name TEXT,
    category INTEGER,
    price REAL
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS clients (
    id_client INTEGER PRIMARY KEY,
    client_name TEXT,
    city TEXT,
    state TEXT
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS sales (
    id_sale INTEGER PRIMARY KEY,
    id_product INTEGER,
    sale_date TEXT,
    quantity INTEGER,
    id_client INTEGER,
    FOREIGN KEY (id_product) REFERENCES products (id_product),
    FOREIGN KEY (id_client) REFERENCES clients (id_client)
)""")

# Querying for tables in the database
cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
print( cursor.fetchall() )

[('products',), ('clients',), ('sales',)]
